In [1]:
import pvl
import struct
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os
import binascii

In [9]:
label_file = "/Users/acpaquette/repos/ISIS3/build/N1472853667_1.LBL"

In [10]:
pvl_label = pvl.load(label_file)
img_file = os.path.join(os.path.split(label_file)[0], pvl_label["^IMAGE"][0])
pvl_label

PVLModule([
  ('PDS_VERSION_ID', 'PDS3')
  ('RECORD_TYPE', 'FIXED_LENGTH')
  ('RECORD_BYTES', 1048)
  ('FILE_RECORDS', 1028)
  ('^IMAGE_HEADER', ['N1472853667_1.IMG', 1])
  ('^TELEMETRY_TABLE', ['N1472853667_1.IMG', 4])
  ('^LINE_PREFIX_TABLE', ['N1472853667_1.IMG', 5])
  ('^IMAGE', ['N1472853667_1.IMG', 5])
  ('ANTIBLOOMING_STATE_FLAG', 'ON')
  ('BIAS_STRIP_MEAN', 17.25049)
  ('CALIBRATION_LAMP_STATE_FLAG', 'N/A')
  ('COMMAND_FILE_NAME', 'trigger_6510_6.ioi')
  ('COMMAND_SEQUENCE_NUMBER', 6510)
  ('DARK_STRIP_MEAN', 0.0)
  ('DATA_CONVERSION_TYPE', 'TABLE')
  ('DATA_SET_ID', 'CO-S-ISSNA/ISSWA-2-EDR-V1.0')
  ('DELAYED_READOUT_FLAG', 'NO')
  ('DESCRIPTION', 'Incomplete product finalized due to truncated lines.')
  ('DETECTOR_TEMPERATURE', Quantity(value=-89.243546, units='DEGC'))
  ('EARTH_RECEIVED_START_TIME',
   datetime.datetime(2004, 9, 3, 9, 54, 37, 232000, tzinfo=datetime.timezone.utc))
  ('EARTH_RECEIVED_STOP_TIME',
   datetime.datetime(2004, 9, 3, 9, 57, 12, 966000, tzinfo=dateti

In [11]:
with open(img_file, 'rb') as fp:
    byte_position = ((pvl_label["^IMAGE"][1] - 1) * pvl_label["RECORD_BYTES"])
    b_original_header = fp.read(byte_position)
    b_image_data = fp.read()

In [12]:
lut_list = [1, 3, 5, 7, 9, 11, 14, 16, 19, 22, 25, 28, 31, 35, 38, 42, 46, 49, 53, 57,
61, 65, 70, 74, 79, 83, 88, 93, 98, 103, 108, 113, 119, 125, 130, 136, 142,
148, 154, 160, 166, 173, 179, 186, 193, 199, 206, 213, 221, 228, 236, 243,
251, 259, 267, 275, 283, 291, 299, 307, 316, 324, 333, 342, 351, 360, 369,
379, 388, 398, 407, 417, 427, 437, 447, 457, 467, 478, 488, 499, 509, 520,
531, 542, 554, 565, 576, 588, 600, 611, 623, 635, 647, 659, 672, 684, 697,
709, 722, 735, 747, 761, 774, 787, 801, 814, 828, 842, 856, 870, 884, 898,
912, 926, 941, 955, 970, 985, 1000, 1015, 1030, 1045, 1061, 1076, 1092, 1108,
1124, 1140, 1156, 1172, 1188, 1204, 1221, 1237, 1254, 1271, 1288, 1305, 1322,
1339, 1357, 1374, 1392, 1410, 1428, 1446, 1464, 1482, 1500, 1518, 1537, 1555,
1574, 1593, 1612, 1631, 1650, 1669, 1689, 1708, 1728, 1748, 1768, 1788, 1808,
1828, 1848, 1868, 1889, 1909, 1930, 1951, 1971, 1993, 2014, 2035, 2057, 2078,
2100, 2121, 2143, 2165, 2187, 2209, 2232, 2254, 2276, 2299, 2322, 2344, 2367,
2390, 2413, 2437, 2460, 2484, 2507, 2531, 2555, 2579, 2603, 2627, 2651, 2676,
2700, 2725, 2749, 2774, 2799, 2824, 2849, 2874, 2900, 2925, 2951, 2977, 3003,
3029, 3055, 3081, 3107, 3133, 3160, 3186, 3213, 3239, 3266, 3293, 3321, 3348,
3375, 3403, 3430, 3458, 3486, 3514, 3542, 3570, 3598, 3627, 3655, 3684, 3712,
3741, 3770, 3799, 3828, 3857, 3887, 3916, 3946, 3976, 4005, 4035, 4065, 4095]

In [13]:
sampling = (pvl_label['IMAGE']['SAMPLE_BITS']//8)
line_length = (pvl_label['IMAGE']['LINE_SAMPLES'] * sampling) + pvl_label['IMAGE']['LINE_PREFIX_BYTES']
print(line_length)
n_lines = 10


image_data = []
for j in range(n_lines):
    start, stop = j*(line_length), (j+1)*(line_length)
    if (sampling == 2):
        dtype = np.int16
    elif (sampling == 4):
        dtype = np.float32
    else:
        dtype=np.int8
    image_sample = np.frombuffer(b_image_data[start:stop], dtype=dtype, count=int((line_length)/sampling))
    image_data.append(image_sample)
image_data = np.array(image_data)

# Uncomment the following to compare with an ISIS cube
# if pvl_label["^IMAGE"][0][0] == 'W':
#     image_data = image_data.byteswap()
# else:
#     image_data = np.reshape(np.array(list(map(lambda x: lut_list[x], image_data.flatten()))), (n_lines, line_length))

1048


In [14]:
path_cropped_label = os.path.splitext(label_file)[0] + ".cropped.lbl"
path_cropped_file = os.path.splitext(img_file)[0] + ".cropped.img"
cropped_file = os.path.split(path_cropped_file)[1]

new_label = pvl_label.copy()
new_label["IMAGE"]["LINES"] = n_lines
new_label["^IMAGE"][0] = cropped_file
new_label["^IMAGE_HEADER"][0] = cropped_file
new_label["^TELEMETRY_TABLE"][0] = cropped_file
new_label["^LINE_PREFIX_TABLE"][0] = cropped_file

with open(path_cropped_file, 'wb') as fp:
    fp.write(b_original_header)
    fp.write(image_data)
    
pvl.dump(new_label, path_cropped_label)

4620

In [ ]:
# Below lines used to compare the ISIS cube to the original label since qt 15.0 doesn't work on big sur
# as of 01/28/2022

In [ ]:
cube_file = "/Path/to/test/output.cub"
cube_label = pvl.load(cube_file)

In [ ]:
with open(cube_file, 'rb') as fp:
    fp.seek(cube_label["IsisCube"]["Core"]["StartByte"] - 1)
    b_cube_data = fp.read()

In [ ]:
line_length = cube_label["IsisCube"]["Core"]["Dimensions"]["Samples"] * 2
n_lines = cube_label["IsisCube"]["Core"]["Dimensions"]["Lines"]

cube_data = []
for j in range(n_lines):
    start, stop = j*(line_length), (j+1)*(line_length)
    
    cube_sample = np.frombuffer(b_cube_data[start:stop], dtype=np.int16, count=int(line_length/2))
    cube_data.append(cube_sample)
    
cube_data = np.array(cube_data)
# cube_data = np.where(cube_data == -32764, 4095, cube_data)
cube_data = np.where(cube_data < -32764, 1, cube_data)

In [ ]:
plt.figure(0, figsize=(20, 20))
plt.imshow(cube_data)
plt.show()